# Performance énergétique des logements et niveau de vie des ménages en Ile-de-France¶

#### **Projet Python 2A ENSAE (année 2022-2023)**

#### Franki Nguimatsia Tiofak, Marc jodel Simo Noubissi, Gabriel Sklénard

## **Introduction**

### **Problématique:** 
 
Dans le contexte de la crise énergétique affectant un grand nombre de pays en 2022, il nous est apparu particulièrement intéressant d'étudier le lien entre performance énergétique des logements et niveau de vie des ménages. En effet, dans une période où la facture énergétique s'alourdit et atteint même des plus haut (en particulier pour le gaz), les ménages les plus défavorisés apparaissent plus vulnérables, et ce d'autant plus si leurs logements sont en moyenne plus énergivores que les ménages plus aisés. Notre projet Python cherche précisément à documenter cette vulnérabilité, avec notamment l'idée que la performance énergétique des logements ont un coût et bute nécessairement contre la contrainte budgétaire des ménages, mesurée ici par le niveau de vie médian au niveau de la commune. Qui plus est, un cercle vicieux pourrait s'instaurer: les ménages les moins aisés ont des difficultés à accéder à un logement économe en énergie, ce qui accroît la charge financière du logement et les appauvrit encore davantage. 

Pour avoir une volumétrie de données raisonnable, nous avons opté pour une approche géographique centrée sur l'Ile-de-France: il s'agit donc ici d'analyser la performance énergétique des logements franciliens en lien avec le niveau de vie mesuré dans chaque commune. La visualisation géographique pourrait contribuer à aider les décideurs publics à mieux cibler les éventuelles mesures de soutien à mettre en place pour atténuer le choc de la crise énergétique en Ile-de-France.

### **Principales étapes du projet:**
1. Récupération des données relatives aux diagnostics de performance énergétique (DPE) des logements franciliens (à partir de l'API de l'open data de l'ADEME.
2. Récupération des données de niveau de vie par commune à partir des fichiers de Filosofi 2016 disponibles sur Insee.fr
3. Retraitement des données.
4. Premières statistiques descriptives
5. Visualisations géographiques
6. Modélisation: quantification du rôle explicatif du niveau de vie médian de la commune sur: 
    a. la structure des logements par classe de consommation d'énergie; 
    b. le niveau moyen de consommation d'énergie par logement;
7. Principales limites et perspectives d'approfondissment

### **Principaux résultats et principales limites:**

#### Résulats de l'étude

1. Résultat 1: 
2. Résultat 2:
3. Résultat 3:

#### Limites de l'étude

1. Limite 1: les DPE observés dans chaque commune d'Ile-de-France ne résultent pas d'un échantillonage reposant sur un
plal de sondage garantissant la représentativité statistique des données à l'échelle de chaque commune. On ne dispose
en effet que d'une liste de DPE versés automatiquement dans la base de données de l'ADEME, dès lors qu'un diagnostic est validé numériquement. Par conséquent, on peut très bien imaginer que nos résultats sont affectés par des biais statistiques importants. Le nombre de DPE enregistrés dans une commune dépend du nombre de locations ou ventes de logements ayant eu lieu dans cette commune.
2. Limite 2:
3. Limite 3:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import re
import matplotlib
import plotly.express as px
import seaborn as sns

import sklearn
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import statsmodels.api as sm
import scipy
from scipy.stats import chi2_contingency as chi2_contingency

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

import plotly
import plotly.express as px
from IPython.display import HTML #pour afficher les graphs
# dans une cellule de notebook

In [11]:
# Pour que les graphiques apparaissent dans le Notebook Jupyter:
%matplotlib inline

## **Importation des données retraitées**

In [2]:
pd.options.display.max_colwidth = 50

On importe les données récupérées et traitées dans le notebook récup&retraitement_données.ipynb:

In [3]:
# 1) Dataframe rassemblant tous les dpe réalisés dans les communes d'Ile-de-France (principales caractéristiques relatives 
# à la performance énergétique des logements):
dpe_idf = pd.read_csv("~/work/data_science_project/dpe_IDF.csv")

# 2) Dataframe rassemblant les communes d'IDF avec leur caractéristiques en terme de performance énergétique des logements
# ainsi que quelques données socio-économiques (Filosofi):
Filosofi_dpe_IDF_com= pd.read_csv("~/work/data_science_project/Filosofi_dpe_IDF_com.csv")


/tmp/ipykernel_1166/4118461782.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dpe_idf = pd.read_csv("~/work/data_science_project/dpe_IDF.csv")


### ** Dictionnaire de variables pour le dataframe dpe_idf:**

- code_commune_actualise:
- 

## **I) Analyse exploratoire des données et datavisualisation**

### **A)Analyse de la performance énergétique des logements à l'échelle des communes d'Ile-de-France:**

#### 1) Etape préliminaire: on exclut de l'analyse les communes pour lesquelles le nombre de DPE s'avère trop faible au regard du nombre de ménages fiscaux renseignés dans Filosofi:

L'interprétabilité des résultats est conditionnée par le fait que l'on dispose d'un nombre suffisant de DPE dans chaque commune. Pour tester cette condition, on calcule le ratio "nombre de DPE/nombre de ménages fiscaux" pour chaqye commune, avec l'idée que le nombre de ménages fiscaux est un bon proxi du nombre total de logements dans une commune donnée.

In [7]:
# Calcul du ratio "nombre de DPE/nombre de ménages fiscaux":
Filosofi_dpe_IDF_com["ratio_DPE_menage"]=Filosofi_dpe_IDF_com["nombre_dpe"]/Filosofi_dpe_IDF_com["NBMENFISC16"]
Filosofi_dpe_IDF_com["ratio_DPE_menage"].describe()

count    1013.000000
mean        0.327655
std         0.686188
min         0.007556
25%         0.097826
50%         0.181564
75%         0.312899
max        11.027027
Name: ratio_DPE_menage, dtype: float64

Au vue de la distribution de ce ratio, on peut décider d'exclure les 25% des communes de l'analyse qui présentent un ratio inférieur à 10%:

In [8]:
Filosofi_dpe_IDF_com2=Filosofi_dpe_IDF_com[Filosofi_dpe_IDF_com["ratio_DPE_menage"]>0.1].reset_index()
Filosofi_dpe_IDF_com2.shape

(751, 52)

In [9]:
Filosofi_dpe_IDF_com.shape

(1288, 52)

#### 2) Premières statistiques descriptives sur la performance énergétique des logements dans les communes d'Ile-de-France

##### a) Création d'une indicatrice pour repérer les communes rassemblant le plus de logements énergivores

In [14]:
# Codage de l'indicatrice "est énergivore" (définition 2):
x=Filosofi_dpe_IDF_com2[["classe_consommation_energie_F","classe_consommation_energie_G"]]
x
x["proportion_energivores"]=x.sum(axis=1)
#FG=x[["proportion_energivores"]]
#Filosofi_dpe_IDF_com3=Filosofi_dpe_IDF_com2.merge(FG,left_index=True,right_index=True)
#Filosofi_dpe_IDF_com3.head()

,classe_consommation_energie_F,classe_consommation_energie_G
1,0.142202,0.084862
3,0.137219,0.086737
4,0.126366,0.075820
6,0.146821,0.084763
7,0.095255,0.052408
...,...,...
1281,0.142857,0.000000
1282,0.002976,0.000000
1285,0.010179,0.003215
1286,0.000000,0.000000


#### 3) Quel lien statistique entre performance énergétique des logements d'une commune et le niveau de niveau médian des ménages qui y vivent?

## **II) Modélisation: prédiction de la structure des logements d'une commune en classes de consommation énergétique**

In [ ]:
# Modèle logistique multinomial?
# Variable à expliquer: la structure des logements de la commune en termes de consommation énergétique
# Variables explicatives: le type de logement, la surface moyenne, l'ancienneté du logement, 
# le niveau de vie médian de la commune, milieur rural/milieu urbain, etc.

